In [1]:
import cv2
import numpy as np
import math
print(cv2.__version__)

4.7.0


Ex 1.1

In [2]:
class EuclideanDistTracker:
    def __init__(self):
        # Store the center positions of the cars
        self.center_points = {}
        
        # each time a new car id detected, counter will increase by one
        self.id_count = 0

    def update(self, cars_rect):
        # Cars' bounding boxes and ids
        cars_bbs_ids = []

        # Get center point of newly detected car
        for rect in cars_rect:
            x, y, w, h = rect
            cx = (x + x + w) // 2
            cy = (y + y + h) // 2

            # Check if car was detected already
            car_already_detected = False
            for id, pt in self.center_points.items():
                dist = math.hypot(cx - pt[0], cy - pt[1])

                if dist < 80:
                    self.center_points[id] = (cx, cy)
                    print(self.center_points)
                    cars_bbs_ids.append([x, y, w, h, id])
                    car_already_detected = True
                    break

            # If new car, assign ID to that car
            if car_already_detected is False:
                self.center_points[self.id_count] = (cx, cy)
                cars_bbs_ids.append([x, y, w, h, self.id_count])
                self.id_count += 1

        # Clean the dictionary by center points to remove IDS not used anymore
        new_center_points = {}
        for car in cars_bbs_ids:
            _, _, _, _, car_id = car
            center = self.center_points[car_id]
            new_center_points[car_id] = center

        # Update dictionary with IDs not used removed
        self.center_points = new_center_points.copy()
        return cars_bbs_ids

# Create tracker object
tracker1 = EuclideanDistTracker()

# create background subtractor object
backSub = cv2.createBackgroundSubtractorMOG2()

# kernel for image dilation
kernel = np.ones((4,4),np.uint8)

# Open the video file
video= cv2.VideoCapture('Traffic_Laramie_1.mp4')

# check if the video opened successfully
if (video.isOpened()== False): 
    print("Error opening video file")

# Read the first frame
ret, frame1 = video.read()

# get width and height of video frame
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

# define dimensions for region of interest(ROI) rectangle
roiX = 0
roiY = int(height/2-40)
roiW = width
roiH = int(height/2+40)

# Convert first frame to grayscale
prev_gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

# Loop through the video
while True:
    # Read the next frame
    ret, frame2 = video.read()
    
    # If there is no next frame, break out of the loop
    if not ret:
        break
    
    detections = []
    
    # Convert frame to grayscale
    gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    
    # Perform frame differencing
    frame_diff = cv2.absdiff(prev_gray, gray)
    
    # apply background subtraction to ROI
    fgMask = backSub.apply(frame_diff)
    
    # remove noise
    image = cv2.GaussianBlur(fgMask, (25,25), 0)

    # apply thresholding to create binary image
    thresh = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY)[1]
    
    # image dilation
    dilated = cv2.dilate(thresh,kernel,iterations = 1)
    
    # find contours in binary image
    contours, hierarchy = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # loop through contours
    for cnt in contours:
        # calculate contour area
        ctArea = cv2.contourArea(cnt)

        # draw bounding box around contour
        (x, y, w, h) = cv2.boundingRect(cnt)
        if y > height/2-40 and ctArea > 2500:
            cv2.rectangle(frame2, (x, y), (x + w, y + h), (0, 255, 0), 2)
            detections.append([x, y, w, h])

            
    cars_ids = tracker1.update(detections)
    for car_id in cars_ids:
        x, y, w, h, id = car_id
        cv2.rectangle(frame2, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    
    # draw ROI rectangle on frame
    cv2.rectangle(frame2, (roiX, roiY), (roiX + roiW, roiY + roiH), (0, 0, 255), 2)
    
    # Display the resulting frame
    cv2.imshow('Frame Difference', dilated)
    cv2.imshow('Frame', frame2)
    
    # Wait for user input to exit
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# Release the capture and destroy any OpenCV windows
video.release()
cv2.destroyAllWindows()

{0: (1013, 542)}
{0: (1010, 540)}
{0: (1009, 540)}
{0: (1006, 539)}
{0: (1004, 538)}
{0: (1000, 535)}
{0: (998, 536)}
{0: (989, 526)}
{0: (990, 531)}
{0: (991, 530)}
{0: (988, 528)}
{0: (985, 526)}
{0: (985, 517)}
{0: (980, 522)}
{0: (977, 518)}
{0: (974, 516)}
{0: (971, 515)}
{0: (970, 508)}
{0: (966, 506)}
{0: (962, 505)}
{0: (959, 502)}
{0: (957, 500)}
{0: (956, 498)}
{0: (944, 496)}
{0: (937, 494)}
{0: (931, 494)}
{0: (927, 497)}
{0: (923, 490)}
{0: (914, 491)}
{0: (909, 489)}
{0: (906, 488)}
{0: (901, 478)}
{0: (899, 477)}
{0: (884, 475)}
{0: (879, 473)}
{0: (872, 472)}
{0: (869, 476)}
{0: (866, 469)}
{0: (857, 470)}
{0: (849, 469)}
{0: (843, 468)}
{0: (837, 461)}
{0: (834, 460)}
{0: (821, 461)}
{0: (814, 461)}
{0: (806, 459)}
{0: (800, 458)}
{0: (798, 457)}
{0: (787, 454)}
{0: (778, 453)}
{0: (771, 451)}
{0: (766, 449)}
{0: (763, 449)}
{0: (747, 445)}
{0: (742, 444)}
{0: (734, 443)}
{0: (728, 441)}
{0: (724, 441)}
{0: (712, 437)}
{0: (704, 436)}
{0: (698, 435)}
{0: (690, 434)}
{0

{1: (253, 491), 3: (563, 429), 4: (36, 368)}
{1: (253, 491), 3: (553, 428), 4: (36, 368)}
{1: (253, 491), 3: (553, 428), 4: (35, 371)}
{1: (258, 492), 3: (553, 428), 4: (35, 371)}
{1: (258, 492), 3: (544, 427), 4: (35, 371)}
{1: (258, 492), 3: (544, 427), 4: (35, 374)}
{1: (263, 493), 3: (544, 427), 4: (35, 374)}
{1: (263, 493), 3: (534, 426), 4: (35, 374)}
{1: (263, 493), 3: (534, 426), 4: (33, 376)}
{1: (263, 493), 3: (534, 426), 4: (33, 376)}
{1: (263, 493), 3: (532, 426), 4: (33, 376)}
{1: (263, 493), 3: (532, 426), 4: (33, 376)}
{1: (271, 494), 3: (532, 426), 4: (33, 376)}
{1: (271, 494), 3: (515, 425), 4: (33, 376)}
{1: (271, 494), 3: (515, 425), 4: (31, 377)}
{1: (274, 495), 3: (515, 425), 4: (31, 377)}
{1: (274, 495), 3: (506, 424), 4: (31, 377)}
{1: (274, 495), 3: (506, 424), 4: (30, 378)}
{1: (281, 496), 3: (506, 424), 4: (30, 378)}
{1: (281, 496), 3: (497, 423), 4: (30, 378)}
{1: (281, 496), 3: (497, 423), 4: (29, 380)}
{1: (285, 497), 3: (497, 423), 4: (29, 380)}
{1: (285, 

{1: (444, 515), 4: (131, 422)}
{1: (444, 515), 4: (135, 423)}
{1: (446, 515), 4: (135, 423)}
{1: (446, 515), 4: (135, 423)}
{1: (448, 515), 4: (135, 423)}
{1: (448, 515), 4: (135, 423)}
{1: (450, 515), 4: (135, 423)}
{1: (450, 515), 4: (136, 422)}
{1: (452, 515), 4: (136, 422)}
{1: (452, 515), 4: (136, 422)}
{1: (453, 515), 4: (136, 422)}
{1: (453, 515), 4: (136, 422)}
{1: (454, 515), 4: (136, 422)}
{1: (454, 515), 4: (136, 422)}
{1: (455, 515), 4: (136, 422)}
{1: (455, 515), 4: (136, 422)}
{1: (456, 515), 4: (136, 422)}
{1: (456, 515), 4: (136, 422)}
{1: (458, 515), 4: (136, 422)}
{1: (458, 515), 4: (136, 422)}
{1: (461, 515), 4: (136, 422)}
{1: (461, 515), 4: (136, 423)}
{1: (462, 515), 4: (136, 423)}
{1: (462, 515), 4: (136, 423)}
{1: (464, 515), 4: (136, 423)}
{1: (464, 515), 4: (136, 423)}
{1: (467, 515), 4: (136, 423)}
{1: (467, 515), 4: (136, 424)}
{1: (469, 515), 4: (136, 424)}
{1: (469, 515), 4: (136, 424)}
{1: (471, 515), 4: (136, 424)}
{1: (471, 515), 4: (136, 425)}
{1: (472

{7: (536, 521), 8: (249, 474)}
{7: (536, 521), 8: (273, 483)}
{7: (536, 521), 8: (273, 483)}
{7: (536, 521), 8: (274, 483)}
{7: (537, 521), 8: (274, 483)}
{7: (537, 521), 8: (274, 483)}
{7: (538, 521), 8: (274, 483)}
{7: (538, 521), 8: (274, 483)}
{7: (540, 521), 8: (274, 483)}
{7: (540, 521), 8: (251, 474)}
{7: (540, 521), 8: (251, 474)}
{7: (540, 521), 8: (275, 483)}
{7: (542, 521), 8: (275, 483)}
{7: (542, 521), 8: (252, 475)}
{7: (543, 521), 8: (252, 475)}
{7: (543, 521), 8: (252, 475)}
{7: (555, 521), 8: (252, 475)}
{7: (555, 521), 8: (253, 475)}
{7: (556, 521), 8: (253, 475)}
{7: (556, 521), 8: (253, 475)}
{7: (559, 520), 8: (253, 475), 9: (24, 417)}
{7: (559, 520), 8: (253, 475), 9: (24, 417)}
{7: (559, 520), 8: (253, 475), 9: (25, 416)}
{7: (560, 520), 8: (253, 475), 9: (25, 416)}
{7: (560, 520), 8: (253, 475), 9: (25, 416)}
{7: (560, 520), 8: (253, 475), 9: (26, 417)}
{7: (560, 520), 8: (253, 475), 9: (26, 417)}
{7: (560, 520), 8: (253, 475), 9: (26, 417)}
{7: (560, 520), 8: (

{7: (731, 521), 10: (321, 486)}
{7: (731, 521), 10: (322, 486)}
{7: (732, 521), 10: (322, 486)}
{7: (732, 521), 10: (322, 486)}
{7: (736, 521), 10: (322, 486)}
{7: (736, 521), 10: (315, 485)}
{7: (738, 521), 10: (315, 485)}
{7: (738, 521), 10: (328, 487)}
{7: (740, 521), 10: (328, 487)}
{7: (740, 521), 10: (330, 487)}
{7: (742, 521), 10: (330, 487)}
{7: (742, 521), 10: (332, 488)}
{7: (743, 521), 10: (332, 488)}
{7: (743, 521), 10: (338, 488)}
{7: (746, 521), 10: (338, 488)}
{7: (746, 521), 10: (330, 486)}
{7: (749, 521), 10: (330, 486)}
{7: (749, 521), 10: (315, 485)}
{7: (753, 520), 10: (315, 485)}
{7: (753, 520), 10: (334, 487)}
{7: (755, 520), 10: (334, 487)}
{7: (755, 520), 10: (337, 487)}
{7: (756, 520), 10: (337, 487)}
{7: (756, 520), 10: (337, 487)}
{7: (760, 520), 10: (337, 487)}
{7: (760, 520), 10: (340, 489)}
{7: (763, 520), 10: (340, 489)}
{7: (763, 520), 10: (341, 489)}
{7: (764, 520), 10: (341, 489)}
{7: (764, 520), 10: (324, 468)}
{7: (765, 520), 10: (324, 468)}
{7: (765

...

{49: (914, 472), 48: (551, 409)}
{49: (914, 472), 48: (542, 409)}
{49: (908, 469), 48: (542, 409)}
{49: (908, 469), 48: (537, 409)}
{49: (902, 469), 48: (537, 409)}
{49: (902, 469), 48: (531, 409)}
{49: (896, 468), 48: (531, 409)}
{49: (896, 468), 48: (525, 409)}
{49: (896, 466), 48: (525, 409)}
{49: (896, 466), 48: (526, 409)}
{49: (888, 465), 48: (526, 409)}
{49: (888, 465), 48: (517, 408)}
{49: (881, 464), 48: (517, 408)}
{49: (881, 464), 48: (513, 407)}
{49: (876, 464), 48: (513, 407)}
{49: (876, 464), 48: (506, 407)}
{49: (870, 461), 48: (506, 407)}
{49: (870, 461), 48: (501, 406)}
{49: (872, 461), 48: (501, 406)}
{49: (872, 461), 48: (500, 405)}
{49: (862, 458), 48: (500, 405)}
{49: (862, 458), 48: (492, 405)}
{49: (856, 457), 48: (492, 405)}
{49: (856, 457), 48: (487, 405)}
{49: (850, 457), 48: (487, 405)}
{49: (850, 457), 48: (481, 405)}
{49: (845, 456), 48: (481, 405)}
{49: (845, 456), 48: (475, 405)}
{49: (845, 455), 48: (475, 405)}
{49: (845, 455), 48: (475, 404)}
{49: (835,

{49: (416, 408), 48: (151, 388), 50: (948, 351)}
{49: (416, 408), 48: (147, 389), 50: (948, 351)}
{49: (416, 408), 48: (147, 389), 50: (944, 348)}
{49: (415, 407), 48: (147, 389), 50: (944, 348)}
{49: (415, 407), 48: (147, 388), 50: (944, 348)}
{49: (415, 407), 48: (147, 388), 50: (944, 348)}
{49: (407, 407), 48: (147, 388), 50: (944, 348)}
{49: (407, 407), 48: (143, 387), 50: (944, 348)}
{49: (407, 407), 48: (143, 387), 50: (940, 344)}
{49: (402, 407), 48: (143, 387), 50: (940, 344)}
{49: (402, 407), 48: (140, 387), 50: (940, 344)}
{49: (402, 407), 48: (140, 387), 50: (937, 341)}
{49: (396, 407), 48: (140, 387), 50: (937, 341)}
{49: (396, 407), 48: (136, 386), 50: (937, 341)}
{49: (396, 407), 48: (136, 386), 50: (934, 338)}
{49: (389, 406), 48: (136, 386), 50: (934, 338)}
{49: (389, 406), 48: (133, 386), 50: (934, 338)}
{49: (389, 406), 48: (133, 386), 50: (931, 335)}
{49: (389, 406), 48: (133, 386), 50: (931, 335)}
{49: (389, 406), 48: (133, 386), 50: (931, 335)}
{49: (389, 406), 48:

{52: (460, 511), 53: (421, 488)}
{52: (457, 512)}
{52: (462, 511)}
{52: (455, 511)}
{52: (467, 511)}
{52: (468, 511)}
{52: (470, 512)}
{52: (473, 512)}
{52: (455, 511)}
{52: (460, 511)}
{52: (477, 511)}
{52: (490, 511)}
{52: (494, 511)}
{52: (496, 511)}
{52: (486, 511)}
{52: (486, 511)}
{52: (506, 512)}
{52: (505, 512)}
{52: (507, 512)}
{52: (515, 512)}
{52: (515, 512)}
{52: (528, 513)}
{52: (528, 513)}
{52: (534, 513)}
{52: (540, 513)}
{52: (540, 514)}
{52: (549, 514)}
{52: (553, 513)}
{52: (560, 513)}
{52: (567, 514)}
{52: (568, 514)}
{52: (576, 514)}
{52: (580, 515)}
{52: (589, 515)}
{52: (595, 515)}
{52: (594, 515)}
{52: (604, 516)}
{52: (600, 516)}
{52: (612, 516)}
{52: (619, 517)}
{52: (621, 516)}
{52: (629, 516)}
{52: (635, 517)}
{52: (644, 516)}
{52: (653, 517)}
{52: (653, 517)}
{52: (663, 516)}
{52: (669, 517)}
{52: (678, 516)}
{52: (687, 517)}
{52: (687, 517)}
{52: (699, 517)}
{52: (705, 517), 54: (187, 470)}
{52: (705, 517), 54: (207, 471)}
{52: (714, 517), 54: (207, 471)}
{

{56: (128, 447), 57: (783, 430)}
{56: (128, 447), 57: (788, 427)}
{56: (130, 447), 57: (788, 427)}
{56: (130, 447), 57: (787, 426)}
{56: (132, 447), 57: (787, 426)}
{56: (132, 447), 57: (793, 420)}
{56: (161, 447), 57: (793, 420)}
{56: (161, 447), 57: (796, 419)}
{56: (162, 446), 57: (796, 419)}
{56: (162, 446), 57: (799, 417)}
{56: (163, 446), 57: (799, 417)}
{56: (163, 446), 57: (803, 413)}
{56: (143, 446), 57: (803, 413)}
{56: (143, 446), 57: (803, 413)}
{56: (175, 460), 57: (803, 413)}
{56: (175, 460), 57: (808, 408)}
{56: (178, 462), 57: (808, 408)}
{56: (178, 462), 57: (812, 405)}
{56: (183, 462), 57: (812, 405)}
{56: (183, 462), 57: (815, 402)}
{56: (186, 463), 57: (815, 402)}
{56: (186, 463), 57: (819, 398)}
{56: (186, 463), 57: (819, 398)}
{56: (186, 463), 57: (817, 398)}
{56: (192, 464), 57: (817, 398)}
{56: (192, 464), 57: (821, 393)}
{56: (195, 465), 57: (821, 393)}
{56: (195, 465), 57: (823, 391)}
{56: (200, 467), 57: (823, 391)}
{56: (200, 467), 57: (826, 389)}
{56: (203,

{61: (443, 368), 60: (916, 360)}
{61: (443, 368), 60: (916, 355)}
{61: (440, 367), 60: (916, 355)}
{61: (440, 367), 60: (916, 354)}
{61: (437, 368), 60: (916, 354)}
{61: (437, 368), 60: (916, 353)}
{61: (435, 368), 60: (916, 353)}
{61: (435, 368), 60: (916, 350)}
{61: (432, 368), 60: (916, 350)}
{61: (432, 368), 60: (916, 350)}
{61: (427, 368), 60: (916, 350)}
{61: (427, 368), 60: (917, 348)}
{61: (425, 368), 60: (917, 348)}
{61: (425, 368), 60: (917, 347)}
{61: (421, 369), 60: (917, 347)}
{61: (421, 369), 60: (916, 344)}
{61: (420, 370), 60: (916, 344)}
{61: (420, 370), 60: (915, 342)}
{61: (415, 369), 60: (915, 342)}
{61: (415, 369), 60: (915, 342)}
{61: (413, 369), 60: (915, 342)}
{61: (413, 369), 60: (914, 339)}
{61: (410, 369), 60: (914, 339)}
{61: (410, 369), 60: (914, 337)}
{61: (405, 369), 60: (914, 337)}
{61: (405, 369), 60: (912, 335)}
{61: (403, 369), 60: (912, 335)}
{61: (403, 369), 60: (912, 333)}
{61: (403, 369), 60: (912, 333)}
{61: (403, 369), 60: (912, 333)}
{61: (398,

In [3]:
# Create tracker object
tracker2 = EuclideanDistTracker()

# create background subtractor object
backSub = cv2.createBackgroundSubtractorMOG2()

# kernel for image dilation
kernel = np.ones((4,4),np.uint8)

# Open the video file
video= cv2.VideoCapture('Traffic_Laramie_2.mp4')

# check if the video opened successfully
if (video.isOpened()== False): 
    print("Error opening video file")

# Read the first frame
ret, frame1 = video.read()

# get width and height of video frame
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

# define dimensions for region of interest(ROI) rectangle
roiX = 0
roiY = int(height/2-40)
roiW = width
roiH = int(height/2+40)

# Convert first frame to grayscale
prev_gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

# Loop through the video
while True:
    # Read the next frame
    ret, frame2 = video.read()
    
    # If there is no next frame, break out of the loop
    if not ret:
        break
    
    detections = []
    
    # Convert frame to grayscale
    gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    
    # Perform frame differencing
    frame_diff = cv2.absdiff(prev_gray, gray)
    
    # apply background subtraction to ROI
    fgMask = backSub.apply(frame_diff)
    
    # remove noise
    image = cv2.GaussianBlur(fgMask, (25,25), 0)

    # apply thresholding to create binary image
    thresh = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY)[1]
    
    # image dilation
    dilated = cv2.dilate(thresh,kernel,iterations = 1)
    
    # find contours in binary image
    contours, hierarchy = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # loop through contours
    for cnt in contours:
        # calculate contour area
        ctArea = cv2.contourArea(cnt)

        # draw bounding box around contour
        (x, y, w, h) = cv2.boundingRect(cnt)
        if y > height/2-40 and ctArea > 2500:
            cv2.rectangle(frame2, (x, y), (x + w, y + h), (0, 255, 0), 2)
            detections.append([x, y, w, h])

            
    cars_ids = tracker2.update(detections)
    for car_id in cars_ids:
        x, y, w, h, id = car_id
        cv2.rectangle(frame2, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    
    # draw ROI rectangle on frame
    cv2.rectangle(frame2, (roiX, roiY), (roiX + roiW, roiY + roiH), (0, 0, 255), 2)
    
    # Display the resulting frame
    cv2.imshow('Frame Difference', dilated)
    cv2.imshow('Frame', frame2)
    
    # Wait for user input to exit
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# Release the capture and destroy any OpenCV windows
video.release()
cv2.destroyAllWindows()

{0: (1018, 475)}
{0: (1017, 472)}
{0: (1015, 464)}
{0: (1014, 462)}
{0: (1013, 455)}
{0: (1011, 451)}
{0: (1010, 448)}
{0: (1008, 445)}
{0: (1007, 443)}
{0: (1005, 437)}
{0: (1004, 434)}
{0: (1002, 429)}
{0: (1001, 427)}
{0: (1001, 425)}
{0: (1000, 420)}
{0: (999, 416)}
{0: (997, 413)}
{0: (996, 410)}
{0: (995, 408)}
{0: (989, 403)}
{0: (987, 400)}
{0: (984, 396)}
{0: (982, 394)}
{0: (980, 392)}
{0: (975, 387)}
{0: (972, 384)}
{0: (970, 380)}
{0: (968, 378)}
{0: (966, 377)}
{0: (962, 373)}
{0: (960, 369)}
{0: (956, 367)}
{0: (955, 365)}
{0: (954, 364)}
{0: (950, 359)}
{0: (947, 356)}
{0: (945, 354)}
{0: (943, 352)}
{0: (941, 350)}
{0: (937, 346)}
{0: (934, 343)}
{0: (932, 341)}
{0: (931, 338)}
{0: (930, 337)}
{0: (926, 333)}
{0: (925, 331)}
{0: (924, 329)}
{0: (921, 327)}
{0: (920, 326)}
{0: (916, 322)}
{0: (914, 320)}
{0: (912, 318)}
{0: (911, 317)}
{0: (911, 315)}
{0: (909, 311)}
{0: (903, 310)}
{0: (900, 308)}
{0: (899, 307)}
{0: (897, 306)}
{0: (894, 302)}
{0: (891, 300)}
{0: (889,

{1: (379, 529), 3: (38, 425), 2: (257, 401)}
{1: (379, 529), 3: (39, 427), 2: (257, 401)}
{1: (379, 529), 3: (39, 427), 2: (250, 400)}
{1: (381, 529), 3: (39, 427), 2: (250, 400)}
{1: (381, 529), 3: (41, 428), 2: (250, 400)}
{1: (381, 529), 3: (41, 428), 2: (242, 400)}
{1: (384, 530), 3: (41, 428), 2: (242, 400)}
{1: (384, 530), 3: (43, 430), 2: (242, 400)}
{1: (384, 530), 3: (43, 430), 2: (238, 400)}
{1: (392, 530), 3: (43, 430), 2: (238, 400)}
{1: (392, 530), 3: (45, 431), 2: (238, 400)}
{1: (392, 530), 3: (45, 431), 2: (228, 397)}
{1: (392, 530), 3: (45, 431), 2: (228, 397)}
{1: (392, 530), 3: (46, 432), 2: (228, 397)}
{1: (392, 530), 3: (46, 432), 2: (223, 396)}
{1: (394, 530), 3: (46, 432), 2: (223, 396)}
{1: (394, 530), 3: (47, 433), 2: (223, 396)}
{1: (394, 530), 3: (47, 433), 2: (218, 395)}
{1: (396, 530), 3: (47, 433), 2: (218, 395)}
{1: (396, 530), 3: (49, 434), 2: (218, 395)}
{1: (396, 530), 3: (49, 434), 2: (212, 389)}
{1: (397, 530), 3: (49, 434), 2: (212, 389)}
{1: (397, 

{10: (414, 529), 6: (962, 520), 11: (1011, 412)}
{10: (351, 537), 6: (962, 520), 11: (1011, 412)}
{10: (351, 537), 6: (965, 515), 11: (1011, 412)}
{10: (351, 537), 6: (965, 515), 11: (1010, 412)}
{10: (357, 538), 6: (965, 515), 11: (1010, 412)}
{10: (357, 538), 6: (968, 515), 11: (1010, 412)}
{10: (357, 538), 6: (968, 515), 11: (1007, 412)}
{10: (369, 538), 6: (968, 515), 11: (1007, 412)}
{10: (369, 538), 6: (976, 513), 11: (1007, 412)}
{10: (369, 538), 6: (976, 513), 11: (1005, 412)}
{10: (383, 539), 6: (976, 513), 11: (1005, 412)}
{10: (383, 539), 6: (979, 511), 11: (1005, 412)}
{10: (383, 539), 6: (979, 511), 11: (1002, 413)}
{10: (395, 550), 6: (979, 511), 11: (1002, 413)}
{10: (341, 512), 6: (979, 511), 11: (1002, 413)}
{10: (341, 512), 6: (985, 509), 11: (1002, 413)}
{10: (341, 512), 6: (985, 509), 11: (1000, 413)}
{10: (387, 541), 6: (985, 509), 11: (1000, 413)}
{10: (387, 541), 6: (990, 508), 11: (1000, 413)}
{10: (387, 541), 6: (990, 508), 11: (998, 414)}
{10: (401, 542), 6: (

{13: (224, 484), 11: (422, 424)}
{13: (224, 484), 11: (418, 425)}
{13: (233, 486), 11: (418, 425)}
{13: (233, 486), 11: (410, 424)}
{13: (231, 486), 11: (410, 424)}
{13: (231, 486), 11: (400, 423)}
{13: (237, 486), 11: (400, 423)}
{13: (237, 486), 11: (391, 423)}
{13: (238, 486), 11: (391, 423)}
{13: (238, 486), 11: (383, 423)}
{13: (253, 488), 11: (383, 423)}
{13: (253, 488), 11: (379, 422)}
{13: (244, 487), 11: (379, 422)}
{13: (244, 487), 11: (371, 422)}
{13: (313, 461), 11: (371, 422)}
{13: (313, 461)}
{13: (310, 461)}
{13: (312, 460)}
{13: (311, 463)}
{13: (307, 464)}
{13: (304, 465)}
{13: (304, 465)}
{13: (300, 466)}
{13: (303, 466)}
{13: (312, 536)}
{13: (344, 532), 14: (290, 446)}
{13: (344, 532), 14: (288, 446)}
{13: (343, 532), 14: (288, 446)}
{13: (343, 532), 14: (297, 457)}
{13: (307, 466), 14: (297, 457)}
{13: (306, 467)}
{13: (290, 432), 15: (360, 527)}
{15: (306, 468), 13: (290, 432)}
{15: (303, 516)}
{15: (348, 509)}
{15: (351, 511), 16: (242, 410)}
{15: (351, 511), 16:

{18: (292, 417)}
{18: (282, 416)}
{18: (275, 415)}
{18: (266, 413)}
{18: (261, 413)}
{18: (254, 412)}
{18: (243, 411)}
{18: (237, 410)}
{18: (231, 409)}
{18: (225, 408)}
{18: (220, 407)}
{18: (212, 406)}
{18: (206, 406)}
{18: (199, 405)}
{18: (192, 404)}
{18: (188, 403)}
{18: (180, 401)}
{18: (174, 400)}
{18: (168, 399)}
{18: (162, 399)}
{18: (157, 398)}
{18: (149, 397)}
{18: (145, 396)}
{18: (138, 395)}
{18: (133, 394)}
{18: (130, 393)}
{18: (121, 392)}
{18: (116, 392)}
{18: (112, 391)}
{18: (108, 390)}
{18: (104, 389)}
{18: (97, 388)}
{18: (92, 387)}
{18: (22, 422)}
{18: (88, 386)}
{18: (23, 423)}
{18: (84, 386)}
{18: (24, 422)}
{18: (80, 385)}
{18: (59, 408)}
{18: (57, 407)}
{18: (54, 408)}
{18: (53, 408)}
{18: (51, 408)}
{18: (47, 408)}
{18: (44, 408)}
{18: (42, 409)}
{18: (39, 409)}
{18: (39, 410)}
{18: (37, 410)}
{18: (39, 411)}
{18: (41, 411)}
{18: (42, 411)}
{18: (43, 411)}
{18: (45, 412)}
{18: (46, 412)}
{18: (48, 413)}
{18: (49, 413)}
{18: (51, 414)}
{18: (53, 415)}
{18: (55,

{18: (479, 546), 20: (105, 445), 19: (941, 306)}
{18: (479, 546), 20: (108, 447), 19: (941, 306)}
{18: (479, 546), 20: (108, 447), 19: (940, 306)}
{18: (481, 546), 20: (108, 447), 19: (940, 306)}
{18: (481, 546), 20: (110, 448), 19: (940, 306)}
{18: (481, 546), 20: (110, 448), 19: (938, 304)}
{18: (490, 547), 20: (110, 448), 19: (938, 304)}
{18: (490, 547), 20: (111, 447), 19: (938, 304)}
{18: (490, 547), 20: (111, 447), 19: (937, 304)}
{18: (493, 547), 20: (111, 447), 19: (937, 304)}
{18: (493, 547), 20: (114, 448), 19: (937, 304)}
{18: (493, 547), 20: (114, 448), 19: (936, 304)}
{18: (493, 547), 20: (114, 448), 19: (936, 304)}
{18: (493, 547), 20: (117, 449), 19: (936, 304)}
{18: (493, 547), 20: (117, 449), 19: (933, 302)}
{18: (514, 547), 20: (117, 449), 19: (933, 302)}
{18: (514, 547), 20: (120, 450), 19: (933, 302)}
{18: (514, 547), 20: (120, 450), 19: (931, 300)}
{18: (517, 547), 20: (120, 450), 19: (931, 300)}
{18: (517, 547), 20: (123, 450), 19: (931, 300)}
{18: (517, 547), 20:

{22: (999, 343)}
{22: (998, 343)}
{22: (998, 343)}
{22: (997, 342)}
{22: (996, 342)}
{22: (995, 342)}
{22: (995, 342)}
{22: (994, 342)}
{22: (993, 342)}
{22: (993, 342)}
{22: (992, 342)}
{22: (991, 342)}
{22: (990, 342)}
{22: (990, 341)}
{22: (989, 341)}
{22: (988, 340)}
{22: (987, 340)}
{22: (986, 343)}
{22: (985, 342)}
{22: (985, 340)}
{22: (984, 338)}
{22: (983, 337)}
{22: (982, 337)}
{22: (981, 336)}
{22: (980, 335)}
{22: (979, 334)}
{22: (978, 334)}
{22: (972, 333)}
{22: (969, 332)}
{22: (968, 331)}
{22: (965, 330)}
{22: (963, 330)}
{22: (962, 329)}
{22: (958, 326)}
{22: (957, 326)}
{22: (955, 324)}
{22: (954, 323)}
{22: (952, 323)}
{22: (949, 320)}
{22: (947, 320)}
{22: (945, 319), 23: (22, 413)}
{23: (23, 413), 22: (945, 319)}
{23: (23, 413), 22: (943, 318)}
{23: (23, 414), 22: (943, 318)}
{23: (23, 414), 22: (942, 317)}
{23: (25, 415), 22: (942, 317)}
{23: (25, 415), 22: (938, 315)}
{23: (25, 414), 22: (938, 315)}
{23: (25, 414), 22: (937, 314)}
{23: (26, 415), 22: (937, 314)}


{26: (999, 533)}
{26: (1001, 539)}
{26: (1001, 543)}
{26: (1005, 546)}
{26: (1007, 550)}
{26: (1010, 550)}
{26: (1012, 558)}
